In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from dlomix.constants import ALPHABET_UNMOD
from dlomix.layers.attention import AttentionLayer
from prosit_t.layers import (
    MetaEncoder,
    FusionLayer,
    TransformerDecoder,
    Regressor,
    SequenceEncoder,
    SequenceEncoderNoGRU,
)
from prosit_t.models import PrositTransformerIntensityPredictorV2
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
from dlomix.data import IntensityDataset
from dlomix.constants import ALPHABET_UNMOD

2023-06-01 16:04:53.280567: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 16:04:54.146084: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
TRAIN_DATAPATH = "https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/Intensity/proteomeTools_train_val.csv"
EPOCHS=1

In [4]:
BATCH_SIZE = 64
int_data = IntensityDataset(
    data_source=TRAIN_DATAPATH,
    seq_length=30,
    collision_energy_col="collision_energy",
    batch_size=BATCH_SIZE,
    val_ratio=0.2,
    test=False,
)

2023-06-01 16:05:04.011450: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 28702 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:21:00.0, compute capability: 8.6
2023-06-01 16:05:04.012022: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 43610 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6
2023-06-01 16:05:04.012390: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 43610 MB memory:  -> device: 2, name: NVIDIA A40, pci bus id: 0000:e2:00.0, compute capability: 8.6


In [5]:
model = PrositTransformerIntensityPredictorV2()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=masked_spectral_distance,
    metrics=[masked_pearson_correlation_distance],
)

16


In [6]:
model.fit(
    int_data.train_data,
    validation_data=int_data.val_data,
    epochs=EPOCHS
)

2023-06-01 16:05:04.694677: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [23601,1]
	 [[{{node Placeholder/_1}}]]
2023-06-01 16:05:04.694895: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [23601,6]
	 [[{{node Placeholder/_2}}]]


(None, 16)
(None, 30, 16) ????
(None, 16) atttt
(None, 16)
(None, 30, 16) ????
(None, 16) atttt


2023-06-01 16:05:13.108542: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-01 16:05:13.192874: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x3a64bfc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-01 16:05:13.192897: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2023-06-01 16:05:13.192903: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (1): NVIDIA A40, Compute Capability 8.6
2023-06-01 16:05:13.192908: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (2): NVIDIA A40, Compute Capability 8.6
2023-06-01 16:05:13.196322: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-01 16:05:

367/369 [============================>.] - ETA: 0s - loss: 0.6592 - masked_pearson_correlation_distance: 0.5427(None, 16)
(None, 30, 16) ????


2023-06-01 16:05:28.246743: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5900]
	 [[{{node Placeholder/_0}}]]
2023-06-01 16:05:28.246977: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [5900,1]
	 [[{{node Placeholder/_1}}]]


(None, 16) atttt
369/369 [==============================] - 25s 21ms/step - loss: 0.6591 - masked_pearson_correlation_distance: 0.5425 - val_loss: 0.6119 - val_masked_pearson_correlation_distance: 0.4737


In [7]:
model.summary()

Model: "prosit_transformer_intensity_predictor_v2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 string_lookup (StringLookup  multiple                 0         
 )                                                               
                                                                 
 embedding (Embedding)       multiple                  352       
                                                                 
 meta_encoder (MetaEncoder)  multiple                  128       
                                                                 
 sequence_encoder_no_gru (Se  multiple                 54192     
 quenceEncoderNoGRU)                                             
                                                                 
 encoder_att (AttentionLayer  multiple                 46        
 )                                                               
                         